In [182]:
import pandas as pd
from pandas import DataFrame, Series

## Create a client

In [183]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

## Get a database

In [184]:
db = client.test_database
db = client['test_database']

## Get a collection

In [185]:
collection = db.test_collection
collection = db['test_collection']

Collections and databases are created lazily

## Documents

In [186]:
import datetime
post = {
    "author": "Mike",
    "text": "My first blog post!",
    "tags": ["mongodb", "python", "pymongo"],
    "date": datetime.datetime.utcnow()
}

Data in mongodb is represented as JSON-style documents. In PyMongo, we use dict to represent documents.

## Insert a document

In [187]:
posts = collection['posts']
posts.delete_many({})

In [188]:
inserted = posts.insert_one(post)

### Get id

id is unique accross the collection, key '_id' is created automatically for each inserted document

In [189]:
inserted_id = inserted.inserted_id
inserted_id

ObjectId('5b84088a23b3981a671e7822')

In [190]:
type(inserted_id)

bson.objectid.ObjectId

In [191]:
str(inserted_id)

'5b84088a23b3981a671e7822'

### Colletion List
Aftr insertion, collection can be found

In [192]:
db.list_collection_names()

['test_collection.posts']

In [193]:
from pprint import pprint

In [194]:
pprint(posts.find_one())

{'_id': ObjectId('5b84088a23b3981a671e7822'),
 'author': 'Mike',
 'date': datetime.datetime(2018, 8, 27, 14, 19, 54, 522000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [195]:
pprint(posts.find_one({'author':'Mike'}))

{'_id': ObjectId('5b84088a23b3981a671e7822'),
 'author': 'Mike',
 'date': datetime.datetime(2018, 8, 27, 14, 19, 54, 522000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [196]:
pprint(posts.find_one({'_id':inserted_id}))

{'_id': ObjectId('5b84088a23b3981a671e7822'),
 'author': 'Mike',
 'date': datetime.datetime(2018, 8, 27, 14, 19, 54, 522000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [197]:
# not working
pprint(posts.find_one({'_id':inserted_id}))

{'_id': ObjectId('5b84088a23b3981a671e7822'),
 'author': 'Mike',
 'date': datetime.datetime(2018, 8, 27, 14, 19, 54, 522000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [198]:
from bson.objectid import ObjectId

In [199]:
pprint(posts.find_one({'_id':ObjectId(str(inserted_id))}))

{'_id': ObjectId('5b84088a23b3981a671e7822'),
 'author': 'Mike',
 'date': datetime.datetime(2018, 8, 27, 14, 19, 54, 522000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [200]:
new_post = {
    '_id': '123',
    "author": "Mike",
    "text": "My first blog post!",
    "tags": ["mongodb", "python", "pymongo"],
    "date": datetime.datetime.utcnow()
}

In [201]:
inserted = posts.insert_one(new_post)

In [202]:
[r for r in posts.find({'author':'Mike'})]

[{'_id': ObjectId('5b84088a23b3981a671e7822'),
  'author': 'Mike',
  'text': 'My first blog post!',
  'tags': ['mongodb', 'python', 'pymongo'],
  'date': datetime.datetime(2018, 8, 27, 14, 19, 54, 522000)},
 {'_id': '123',
  'author': 'Mike',
  'text': 'My first blog post!',
  'tags': ['mongodb', 'python', 'pymongo'],
  'date': datetime.datetime(2018, 8, 27, 14, 19, 54, 647000)}]

In [203]:
posts.find_one({'_id':'123'})

{'_id': '123',
 'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo'],
 'date': datetime.datetime(2018, 8, 27, 14, 19, 54, 647000)}

In [204]:
new_posts = [{"author": "Mike",
               "text": "Another post!",
               "tags": ["bulk", "insert"],
               "date": datetime.datetime(2009, 11, 12, 11, 14)},
              {"author": "Eliot",
               "title": "MongoDB is fun",
               "text": "and pretty easy too!",
               "date": datetime.datetime(2009, 11, 10, 10, 45)}]

In [205]:
inserted = posts.insert_many(new_posts)

In [206]:
inserted.inserted_ids

[ObjectId('5b84088a23b3981a671e7823'), ObjectId('5b84088a23b3981a671e7824')]

## Count

In [207]:
posts.count_documents({'_id':'123'})

1

In [208]:
d = datetime.datetime(2009, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    pprint(post)

{'_id': ObjectId('5b84088a23b3981a671e7824'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}
{'_id': ObjectId('5b84088a23b3981a671e7823'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


In [210]:
posts.create_index([('author', pymongo.ASCENDING)], unique=False)

'author_1'